In [ ]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install unstructured
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a844a7151334657192c8c8ec8df02cf35847f9e2f

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [ ]:
#load pdf files
#loader = PyPDFDirectoryLoader("/content/drive/MyDrive/LLM/Data")
#data = loader.load()



In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
URLs=[
    'https://www.mosaicml.com/blog/mpt-7b'
    ]
loaders=UnstructuredURLLoader(urls=URLs)
data=loaders.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
print(data)

[Document(page_content='All\n\nCustomer Stories\n\nEcosystem\n\nEngineering\n\nResearch\n\nLaunch\n\nMethodology\n\nMore\n\nLink 1\n\nCustomer Stories\n\nEcosystem\n\nEngineering\n\nResearch\n\nLaunch\n\nMethodology\n\nResearch\n\nby\n\nThe MosaicML NLP Team\n\non\n\nMay 5, 2023\n\nShare\n\nIntroducing MPT-7B: A New Standard for Open-Source, Commercially Usable LLMs\n\nIntroducing MPT-7B, the first entry in our MosaicML Foundation Series. MPT-7B is a transformer trained from scratch on 1T tokens of text and code. It is open source, available for commercial use, and matches the quality of LLaMA-7B. MPT-7B was trained on the MosaicML platform in 9.5 days with zero human intervention at a cost of ~$200k.\n\nLarge language models (LLMs) are changing the world, but for those outside well-resourced industry labs, it can be extremely difficult to train and deploy these models. This has led to a flurry of activity centered on open-source LLMs, such as the LLaMA series from Meta, the Pythia ser

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

34

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content='Today, we at MosaicML are releasing a new model series called MPT (MosaicML Pretrained Transformer) to address the limitations of the above models and finally provide a commercially-usable, open-source model that matches (and - in many ways - surpasses) LLaMA-7B. Now you can train, finetune, and deploy your own private MPT models, either starting from one of our checkpoints or training from scratch. For inspiration, we are also releasing three finetuned models in addition to the base MPT-7B: MPT-7B-Instruct, MPT-7B-Chat, and MPT-7B-StoryWriter-65k+, the last of which uses a context length of 65k tokens!\n\nOur MPT model series is:\n\nLicensed for commercial use (unlike LLaMA).\n\nTrained on a large amount of data (1T tokens like LLaMA vs. 300B for Pythia, 300B for OpenLLaMA, and 800B for StableLM).\n\nPrepared to handle extremely long inputs thanks to ALiBi (we trained on up to 65k inputs and can handle up to 84k vs. 2k-4k for other open source models).', metadat

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from ctransformers import AutoModelForCausalLM

llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGUF",model_file="llama-2-7b-chat.Q2_K.gguf",temperature=0.8)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

ValidationError: ignored

In [ ]:
#query = "what has led to a flurry of activity centered on open-source LLMs?"

In [ ]:
#qa.run(query)

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

In [ ]:
response_schemas = [
    ResponseSchema(name="Test Case", description="Name or number of test case"),
    ResponseSchema(name="Preconditions", description="Condition to check before testing the page"),
    ResponseSchema(name="Summary", description="Summary of the page"),
    ResponseSchema(name="Step Action", description="Step present in the testing of the page"),
    ResponseSchema(name="pass_fail_status", description="pass or fail status of the testcase")

]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="Give multiple test cases for the Webpage.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
_input = prompt.format_prompt(question="Login Page")
output = llm(_input.to_string())
#output_parser.parse(output)

In [ ]:
output

' Test Cases\n\n```json\n{\n\t"Test Case": "Login Page",\n\t"Preconditions": "Already logged in user",\n\t"Summary": "Verify login functionality of the webpage",\n\t"Step Action": "Enter valid username and password",\n\t"pass_fail_status": "Pass"\n}\n```\nSearch Page Test Cases\n\n```json\n{\n\t"Test Case": "Search Page",\n\t"Preconditions": "User with search query",\n\t"Summary": "Verify functionality of searching on the webpage",\n\t"Step Action": "Enter valid search query",\n\t"pass_fail_status": "Pass"\n}\n```\nAdd to Cart Page Test Cases\n\n```json\n{\n\t"Test Case": "Add to Cart Page",\n\t"Preconditions": "Product in shopping cart",\n\t"Summary": "Verify functionality of adding product to cart on webpage",\n\t"Step Action": "Click add to cart button",\n\t"pass_fail_status": "Pass"\n}\n```\nUser Profile Page Test Cases\n\n```json\n{\n\t"'

In [ ]:
# Remove lines with headers like 'Test Cases:' or 'Search Page Test Cases:'
import re
import json
output=output.replace('\n', '').replace('\r', '')
lines =[]
for match in re.findall(r'\{.*?\}', output):
    print(match)
    lines.append(match)
print(lines)
json_lines = [line for line in lines if line.startswith('{')]

# Convert each line of json into a dictionary and store it into a list
json_objects = [json.loads(line) for line in json_lines]

# Store the list of dictionaries into a new json document
with open('test_cases.json', 'w') as outfile:
    json.dump(json_objects, outfile)

{	"Test Case": "Login Page",	"Preconditions": "Already logged in user",	"Summary": "Verify login functionality of the webpage",	"Step Action": "Enter valid username and password",	"pass_fail_status": "Pass"}
{	"Test Case": "Search Page",	"Preconditions": "User with search query",	"Summary": "Verify functionality of searching on the webpage",	"Step Action": "Enter valid search query",	"pass_fail_status": "Pass"}
{	"Test Case": "Add to Cart Page",	"Preconditions": "Product in shopping cart",	"Summary": "Verify functionality of adding product to cart on webpage",	"Step Action": "Click add to cart button",	"pass_fail_status": "Pass"}
['{\t"Test Case": "Login Page",\t"Preconditions": "Already logged in user",\t"Summary": "Verify login functionality of the webpage",\t"Step Action": "Enter valid username and password",\t"pass_fail_status": "Pass"}', '{\t"Test Case": "Search Page",\t"Preconditions": "User with search query",\t"Summary": "Verify functionality of searching on the webpage",\t"Ste

In [ ]:
json_data = json.dumps(json_lives, indent=4)

# Write the JSON string to a file
with open('testCases.json', 'w') as file:
    file.write(json_data)